In [1]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np

In [2]:
with open('notes','rb') as f:
    notes=pickle.load(f)

In [3]:
n_vocab = len(set(notes))

In [4]:
print("Total Notes: ", len(notes))
print("Unique Notes: ", n_vocab)

Total Notes:  59652
Unique Notes:  358


In [5]:
print(notes[:100])

['4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '11+4', '4+9', '11+4', '4+9', '4+9', '4+9', '4+9', '4+9', '0+4', 'E2', '4+9', '0+4', '4+9', '4+9', '4+9', '4+9', '4+9', '9+2', '4+9', '9+2', '9+2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '5+11', '4+9', '5+11', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '9+0', 'E2', '4+9', '9+0', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '11+2', '4+9', '11+2', '11+2', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '3+7+11', 'E-2', '3+7+11', 'B2', 'G2', '1+5+9', 'F#2', '1+5+9', '3+7+11', 'E-2', '3+7+11', 'G2', 'B2', 'E-3']


### Preparing Sequential Data for LSTM

In [6]:
sequence_length = 100 #no of elements LSTM i/p should consider

In [7]:
pitchnames = sorted(set(notes))

In [8]:
#mapping
el_to_int = dict((e,num) for num,e in enumerate(pitchnames))

In [9]:
#el_to_int

In [10]:
network_ip = []
network_op = []

In [11]:
for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    
    network_ip.append([el_to_int[ch] for ch in seq_in])
    network_op.append(el_to_int[seq_out])

In [12]:
len(network_ip[0])

100

In [13]:
#No of examples
n_patterns = len(network_ip)
print(n_patterns)

59552


In [14]:
network_ip = np.reshape(network_ip,(n_patterns,sequence_length,1))
print(network_ip.shape)

(59552, 100, 1)


In [15]:
# Normalizing data
normalized_network_ip = network_ip/float(n_vocab)

In [17]:
# Network output are the classes, encode into one hot vector
from keras.utils import to_categorical

network_op = to_categorical(network_op)
print(network_op.shape)

(59552, 358)


## Create the Model

In [18]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [19]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalized_network_ip.shape[1], normalized_network_ip.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

In [20]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [22]:
checkpoint = ModelCheckpoint("model.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')


hist = model.fit(normalized_network_ip, network_op, epochs=100, batch_size=64, callbacks=[checkpoint])

Epoch 1/100
931/931 [==============================] - 67s 72ms/step - loss: 4.7739
Epoch 2/100
931/931 [==============================] - 70s 75ms/step - loss: 4.7319
Epoch 3/100
931/931 [==============================] - 71s 76ms/step - loss: 4.7248
Epoch 4/100
931/931 [==============================] - 72s 77ms/step - loss: 4.7221
Epoch 5/100
931/931 [==============================] - 72s 78ms/step - loss: 4.7204
Epoch 6/100
931/931 [==============================] - 72s 78ms/step - loss: 4.7194
Epoch 7/100
931/931 [==============================] - 72s 77ms/step - loss: 4.7194
Epoch 8/100
931/931 [==============================] - 72s 77ms/step - loss: 4.7181
Epoch 9/100
931/931 [==============================] - 72s 78ms/step - loss: 4.7175
Epoch 10/100
931/931 [==============================] - 72s 78ms/step - loss: 4.7165
Epoch 11/100
931/931 [==============================] - 72s 77ms/step - loss: 4.7166
Epoch 12/100
931/931 [==============================] - 72s 77ms/step - lo